# IRAK4 IC50 prediction using XGBoost Regression 

### Import

In [2]:
import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.inf, linewidth=99)
import os
import random

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
import xgboost
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

In [236]:
CFG = {
    'NBITS':2048,
    'SEED':42,
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG['SEED']) # Seed 고정

### DataLoad

In [237]:
# SMILES 데이터를 분자 지문으로 변환
def smiles_to_fingerprint(smiles):
    # 먼저 SMILES 데이터로 부터 구조식 데이터를 뽑고,
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # 구조식으로부터 2048개의 0과 1로 이루어진 분자지문 데이터를 객체를 얻는다. 
        # 이 때, 1은 어떤 분자구조의 존재를 의미하며 0과 1의 개수는 nBits에 따라 달라진다. 
        # bitInfo 에 dictionary를 할당하면 어느 인덱스에 지문이 찍혔는지 알 수 있으며,
        # radius는 단일 맵핑 특성 생성에 사용되는 원자들의 반경,
        # nBits는 분자구조 맵핑을 위한 경로들의 고유조합수를 뜻한다.
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=CFG['NBITS'])
        # 객체를 배열로 변환해서 리턴
        return np.array(fp)
    else:
        # 구조식이 데이터가 없는 경우 2048개의 0을 리턴
        return np.zeros((CFG['NBITS'],))

In [7]:
# 학습 ChEMBL 데이터 로드
chembl_data = pd.read_csv('train.csv')  # 예시 파일 이름
chembl_data.head()

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
0,CHEMBL4443947,IC50,'=',0.022,nM,10.66,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,IC50,'=',0.026,nM,10.59,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,IC50,'=',0.078,nM,10.11,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,IC50,'=',0.081,nM,10.09,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,IC50,'=',0.099,nM,10.00,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...


### Data Pre-processing

In [250]:
train = chembl_data[['Smiles', 'pIC50']]
train['Fingerprint'] = train['Smiles'].apply(smiles_to_fingerprint)

# 훈련데이터 X는 위 함수에서 얻은 분자지문 값, Y는 각각에 상응하는 pIC50 값으로 한다
train_x = np.stack(train['Fingerprint'].values)
train_y = train['pIC50'].values

# 학습 및 검증 데이터 분리
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

c:\Users\LG\anaconda3\envs\IRAK4IC50\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Train & Validation

In [9]:
# 의사결정나무류 최고의 모델로 학습
model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7)
model.fit(train_x, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.08, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [510]:
# 의사결정나무류 최고의 모델로 학습
model = xgboost.XGBRegressor(n_estimators=300, max_depth=8, learning_rate=0.09, subsample=0.7, colsample_bytree=0.9)
model.fit(train_x, train_y)
# train r2: 0.9698
# val r2: 0.7256

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.9,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.09, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=300, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [525]:
model = xgboost.XGBRegressor(n_estimators=1000, learning_rate=0.09, verbosity=1, random_state=42, early_stopping_rounds=50,
                          gamma=0.1, importance_type='gain', reg_alpha=0, reg_lambda=1, min_child_weight=0, scale_pos_weight=1,
                          subsample=0.7, colsample_bytree=1, booster='gbtree', silent=1, max_depth=6)
model.fit(train_x, train_y, eval_set=[(val_x, val_y)])
# train r2: 0.9341
# val r2: 0.7278

[09:33:06] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:6.45528
[1]	validation_0-rmse:5.88587
[2]	validation_0-rmse:5.36936
[3]	validation_0-rmse:4.89987
[4]	validation_0-rmse:4.47396
[5]	validation_0-rmse:4.08728
[6]	validation_0-rmse:3.73065
[7]	validation_0-rmse:3.40760
[8]	validation_0-rmse:3.11786
[9]	validation_0-rmse:2.85459
[10]	validation_0-rmse:2.61281
[11]	validation_0-rmse:2.39607
[12]	validation_0-rmse:2.20091
[13]	validation_0-rmse:2.02243
[14]	validation_0-rmse:1.86526
[15]	validation_0-rmse:1.72191
[16]	validation_0-rmse:1.59495
[17]	validation_0-rmse:1.47588
[18]	validation_0-rmse:1.37211
[19]	v

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=500, enable_categorical=False,
             eval_metric=None, gamma=0.1, gpu_id=-1, grow_policy='depthwise',
             importance_type='gain', interaction_constraints='',
             learning_rate=0.09, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=0,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, ...)

In [526]:
# train 데이터로부터의 학습 모델 평가

# pIC50 값 예측
train_y_pred = model.predict(train_x)
# 정답 IC50 값과 예측된 IC50 값을 비교하여 평균 제곱 오차값 산출 
mse = mean_squared_error(pIC50_to_IC50(train_y), pIC50_to_IC50(train_y_pred))
# 평균 제곱 오차값을 평균제곱근 오차값으로 변환
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

# # 훈련데이터와 그 예측값(pIC50)과의 상관관계(결정계수) 
print("r2_score:", r2_score(train_y, train_y_pred))


RMSE: 1627.4641846534028
r2_score: 0.9340548645979841


In [527]:
# Validation 데이터로부터의 학습 모델 평가

# pIC50 값 예측
val_y_pred = model.predict(val_x)
# 정답 IC50 값과 예측된 IC50 값을 비교하여 평균 제곱 오차값 산출 
mse = mean_squared_error(pIC50_to_IC50(val_y), pIC50_to_IC50(val_y_pred))
# 평균 제곱 오차값을 평균제곱근 오차값으로 변환
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

# val 데이터로 결정계수 확인
print("r2_score:", r2_score(val_y, val_y_pred))

RMSE: 2171.3986061769915
r2_score: 0.7278241335885223


### Inference

In [30]:
# test 데이터로부터의 학습 모델 평가
test = pd.read_csv('./test.csv')
test['Fingerprint'] = test['Smiles'].apply(smiles_to_fingerprint)

test_x = np.stack(test['Fingerprint'].values)

# pIC50 값 예측
test_y_pred = model.predict(test_x)

np.set_printoptions(threshold=np.inf, linewidth=np.inf)
pIC50_to_IC50(test_y_pred)[:10]

array([181.96170602,  31.64219967,  10.78052721,  21.3766674 ,  25.31278906,   9.00616709,  26.86828832,  25.04378692,  35.17156285, 180.01139604])

### Submission

In [31]:
submit = pd.read_csv('./sample_submission.csv')
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit.head()

,ID,IC50_nM
0,TEST_000,181.961706
1,TEST_001,31.642200
2,TEST_002,10.780527
3,TEST_003,21.376667
4,TEST_004,25.312789


In [32]:
submit.to_csv('./baseline_submit.csv', index=False)